In [18]:
import json
from tqdm import tqdm

In [27]:
big = "memoryalpha"
small = "stexpanded"
top_k = 1

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/matched_nodes_communities_leiden/top_10_pairs/" + big + "-" + small + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
gold_pairs = []
matched_communities = []

In [28]:
with open(communities_small_file) as csf, open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf:
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [29]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [30]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        node = matched_community[0]
        community = set()
        for i in range(0, top_k):
            community.update(small_communities[matched_community[1][i]])
        if gold_pair[0] == node and gold_pair[1] in community:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:08<00:00, 16.90it/s]

Gold pairs in matched communities top 1: 19.594594594594593%


In [ ]:
found_small = 0
for gold_pair in gold_pairs:
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Small: " + str(found_small / all_pairs * 100) + "%")